In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [0]:
df = spark.sql("select * from databricks_catalog.silver.orders_silver")

In [0]:
df.display()

In [0]:
df_dimcus = spark.sql("select DimCustomerKey, customer_id AS dim_customer_id from databricks_catalog.gold.dimcustomers")
df_dimcus.display()


In [0]:
df_dimpro = spark.sql("select product_id as DimProductKey, product_id as dim_product_id from databricks_catalog.gold.dimproducts")
df_dimpro.display()

In [0]:
%sql
select * from databricks_catalog.gold.dimproducts

In [0]:
df_fact = df.join(df_dimcus,df["customer_id"] == df_dimcus["dim_customer_id"],how='left').join(df_dimpro,df["product_id"] == df_dimpro["dim_product_id"],how='left')



In [0]:

df_fact_new = df_fact.drop("dim_customer_id","dim_product_id",'customer_id','product_id')
df_fact_new.display()

In [0]:
from delta.tables import *


In [0]:
if spark.catalog.tableExists("databricks_catalog.gold.FactOrders"):
    dlt_obj = DeltaTable.forName(spark, "databricks_catalog.gold.FactOrders")
    dlt_obj.alias("t").merge(df_fact_new.alias("s"),"t.order_id = s.order_id and t.DimCustomerKey = s.DimCustomerKey").whenNotMatchedInsertAll().whenMatchedUpdateAll().execute()
else: 
    df_fact_new.write.format("delta").option("path","abfss://gold@databricksetecharif.dfs.core.windows.net/orders").saveAsTable("databricks_catalog.gold.FactOrders")    


In [0]:
%sql
select * from databricks_catalog.gold.factorders